In [2]:
import pandas as pd
import numpy as np
import json
import os

# ==========================================
# 配置相对路径
# ==========================================
GRAVITY_CSV = "../../data_origin/Gravity/Gravity_csv_V202211/Gravity_V202211.csv"
JSON_PATH = "../country_list.json"
OUT_FILE = "../../data/1-3-norm_geographical_proximity_avg.csv"
YEARS = range(2005, 2024)

def get_average_geography_matrix(csv_path: str, meta_path: str, years: range) -> pd.DataFrame:
    with open(meta_path, 'r', encoding='utf-8') as f:
        meta = json.load(f)['countries']
        target_isos = [c['iso'] for c in meta]
    
    # 1. 一次性读取所需列，减少内存占用
    df_all = pd.read_csv(csv_path, usecols=['year', 'iso3_o', 'iso3_d', 'distw_arithmetic'])
    
    yearly_matrices = []

    for yr in years:
        # 筛选当年及 49 国数据
        df_yr = df_all[(df_all['year'] == yr) & 
                       (df_all['iso3_o'].isin(target_isos)) & 
                       (df_all['iso3_d'].isin(target_isos))]
        
        if df_yr.empty:
            print(f"跳过年份 {yr}：数据不存在")
            continue

        # 2. 转换为矩阵并处理重复项 (aggfunc='mean')
        m = df_yr.pivot_table(index='iso3_o', columns='iso3_d', values='distw_arithmetic', aggfunc='mean')
        m = m.reindex(index=target_isos, columns=target_isos)
        
        # 3. 填补极少数可能的缺失值（使用该年矩阵均值或邻近年份）
        if m.isnull().values.any():
            m = m.ffill(axis=0).bfill(axis=0)

        # 4. 年度局部归一化 [0, 1]
        m_log = np.log(m)
        mask = ~np.eye(len(target_isos), dtype=bool) # 排除对角线（自距离）
        
        v_min = m_log.values[mask].min()
        v_max = m_log.values[mask].max()
        
        # 距离归一化 (0=最近, 1=最远)
        m_norm = (m_log - v_min) / (v_max - v_min)
        
        # 转换为邻近度 (1=最近, 0=最远)
        m_prox = 1 - m_norm
        np.fill_diagonal(m_prox.values, 0) # 对角线设为 0
        
        yearly_matrices.append(m_prox)
        print(f"已完成年份归一化: {yr}")

    # 5. 计算跨年平均
    if not yearly_matrices:
        raise ValueError("没有可用的年份数据进行计算！")
        
    avg_matrix = sum(yearly_matrices) / len(yearly_matrices)
    
    return avg_matrix.sort_index(axis=0).sort_index(axis=1)

# ==========================================
# 执行
# ==========================================
if __name__ == "__main__":
    if not os.path.exists(GRAVITY_CSV):
        print(f"错误：找不到文件 {GRAVITY_CSV}")
    else:
        os.makedirs(os.path.dirname(OUT_FILE), exist_ok=True)
        final_matrix = get_average_geography_matrix(GRAVITY_CSV, JSON_PATH, YEARS)
        final_matrix.to_csv(OUT_FILE)
        print(f"--- 任务成功 ---")
        print(f"Layer 3 (多年平均地理邻近矩阵) 已保存至: {OUT_FILE}")

已完成年份归一化: 2005
已完成年份归一化: 2006
已完成年份归一化: 2007
已完成年份归一化: 2008
已完成年份归一化: 2009
已完成年份归一化: 2010
已完成年份归一化: 2011
已完成年份归一化: 2012
已完成年份归一化: 2013
已完成年份归一化: 2014
已完成年份归一化: 2015
已完成年份归一化: 2016
已完成年份归一化: 2017
已完成年份归一化: 2018
已完成年份归一化: 2019
已完成年份归一化: 2020
已完成年份归一化: 2021
跳过年份 2022：数据不存在
跳过年份 2023：数据不存在
--- 任务成功 ---
Layer 3 (多年平均地理邻近矩阵) 已保存至: ../../data/1-3-norm_geographical_proximity_avg.csv


##### 不取对数（最终启用版）

In [ ]:
import pandas as pd
import numpy as np
import json
import os

# ==========================================
# 配置相对路径
# ==========================================
GRAVITY_CSV = "../../data_origin/Gravity/Gravity_csv_V202211/Gravity_V202211.csv"
JSON_PATH = "../country_list.json"
OUT_FILE = "../../data/1-3-norm_geographical_proximity_avg.csv"
YEARS = range(2005, 2024)

def get_average_geography_matrix(csv_path: str, meta_path: str, years: range) -> pd.DataFrame:
    with open(meta_path, 'r', encoding='utf-8') as f:
        meta = json.load(f)['countries']
        target_isos = [c['iso'] for c in meta]
    
    # 1. 一次性读取所需列，减少内存占用
    df_all = pd.read_csv(csv_path, usecols=['year', 'iso3_o', 'iso3_d', 'distw_arithmetic'])
    
    yearly_matrices = []

    for yr in years:
        # 筛选当年及 49 国数据
        df_yr = df_all[(df_all['year'] == yr) & 
                       (df_all['iso3_o'].isin(target_isos)) & 
                       (df_all['iso3_d'].isin(target_isos))]
        
        if df_yr.empty:
            print(f"跳过年份 {yr}：数据不存在")
            continue

        # 2. 转换为矩阵并处理重复项 (aggfunc='mean')
        m = df_yr.pivot_table(index='iso3_o', columns='iso3_d', values='distw_arithmetic', aggfunc='mean')
        m = m.reindex(index=target_isos, columns=target_isos)
        
        # 3. 填补极少数可能的缺失值（使用该年矩阵均值或邻近年份）
        if m.isnull().values.any():
            m = m.ffill(axis=0).bfill(axis=0)

        # 4. 年度局部归一化 [0, 1]
        # === 修改处：去掉 np.log ===
        m_log = m
        mask = ~np.eye(len(target_isos), dtype=bool) # 排除对角线（自距离）
        
        v_min = m_log.values[mask].min()
        v_max = m_log.values[mask].max()
        
        # 距离归一化 (0=最近, 1=最远)
        m_norm = (m_log - v_min) / (v_max - v_min)
        
        # 转换为邻近度 (1=最近, 0=最远)
        m_prox = 1 - m_norm
        np.fill_diagonal(m_prox.values, 0) # 对角线设为 0
        
        yearly_matrices.append(m_prox)
        print(f"已完成年份归一化: {yr}")

    # 5. 计算跨年平均
    if not yearly_matrices:
        raise ValueError("没有可用的年份数据进行计算！")
        
    avg_matrix = sum(yearly_matrices) / len(yearly_matrices)
    
    return avg_matrix.sort_index(axis=0).sort_index(axis=1)

# ==========================================
# 执行
# ==========================================
if __name__ == "__main__":
    if not os.path.exists(GRAVITY_CSV):
        print(f"错误：找不到文件 {GRAVITY_CSV}")
    else:
        os.makedirs(os.path.dirname(OUT_FILE), exist_ok=True)
        final_matrix = get_average_geography_matrix(GRAVITY_CSV, JSON_PATH, YEARS)
        final_matrix.to_csv(OUT_FILE)
        print(f"--- 任务成功 ---")
        print(f"Layer 3 (多年平均地理邻近矩阵) 已保存至: {OUT_FILE}")

已完成年份归一化: 2005
已完成年份归一化: 2006
已完成年份归一化: 2007
已完成年份归一化: 2008
已完成年份归一化: 2009
已完成年份归一化: 2010
已完成年份归一化: 2011
已完成年份归一化: 2012
已完成年份归一化: 2013
已完成年份归一化: 2014
已完成年份归一化: 2015
已完成年份归一化: 2016
已完成年份归一化: 2017
已完成年份归一化: 2018
已完成年份归一化: 2019
已完成年份归一化: 2020
已完成年份归一化: 2021
跳过年份 2022：数据不存在
跳过年份 2023：数据不存在
--- 任务成功 ---
Layer 3 (多年平均地理邻近矩阵) 已保存至: ../../data/1-3-norm_geographical_proximity_avg.csv
